In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm

import os
import time
import itertools

from glove.model import *

In [2]:
# import file names
files = os.listdir("data/")
strains = ["DSM", "MS008", "MS014", "Strain292", "Strain296"]

# define names of species
species = ['BA', 'BO', 'BU', 'PJ', 'ER', 'AC', 
           'CC','CD', 'DP', 'EL', 'BY', 'CG', 
           'CH', 'BL', 'BF', 'BT', 'BV', 'DL',
           'BH', 'RI', 'BP', 'PC', 'DF', 'CA', 
           'BC', 'CS']

In [3]:
files

['DSM_processed_withmono.csv',
 'MS008_processed_withmono.csv',
 'MS014_processed_withmono.csv',
 'Strain292_processed_withmono.csv',
 'Strain296_processed_withmono.csv',
 'Universal_processed_withmono.csv']

# fit gLV models

In [ ]:
exp_names = []
for strain in strains[1:]:
    # import data
    df_universal = pd.read_csv("data/Universal_processed_withmono.csv")
    df_strain = pd.concat([pd.read_csv(f"data/{fname}") for fname in files if strain in fname])
    df = pd.concat((df_universal, df_strain))
    
    # rename treatments with replicate num
    treatment_names = [t+"_"+str(rep) for t, rep in zip(df["Treatment"].values, df["Replicate"].values)]    
    df.sort_values(by=["Treatment", "Time"], inplace=True)
    df.rename(columns={"Treatment":"Treatments"}, inplace=True)

    # instantiate gLV fit 
    model = gLV(species, df)

    # fit to data 
    model.fit()
    
    # list of parameter names 
    param_names = []
    for s1 in species:
        for s2 in species:
            param_names += [s1+"*"+s2]
    param_names = list(species) + param_names
    
    # plot parameter distribution
    n_species = len(species)
    Avec = model.params[n_species:]
    Aij_std = np.sqrt(np.diag(model.Ainv))[n_species:]

    '''plt.figure(figsize=(18,18))
    # set counter for parameter std. 
    k = 0

    for i in range(n_species):
        for j in range(n_species):
            plt.subplot(n_species, n_species, k+1)
            a = np.linspace(Avec[k]-np.std(Avec), Avec[k]+np.std(Avec))
            plt.plot(a, norm.pdf(a,Avec[k],Aij_std[k]))
            plt.axvline(x=0, c='k', alpha=.5)
            k += 1
            if j == 0:
                plt.ylabel(species[i], fontsize=18)
            if i == n_species-1:
                plt.xlabel(species[j], fontsize=18)
            #plt.xlim([-2,2])

    plt.suptitle(strain, fontsize=24)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("figures/"+strain+".pdf")
    plt.close()'''
    
    # compute Wald test for each parameter
    std_errors = np.sqrt(np.diag(model.Ainv))
    walds = model.params/std_errors
    wald_p_vals = 2*norm.cdf(-np.abs(walds))

    # save to df 
    df = pd.DataFrame()
    df["Param name"] = param_names
    df["Param value"] = model.params
    df["Param stdv"]  = np.sqrt(np.diag(model.Ainv))
    df["Param p-value"] = wald_p_vals
    df = pd.concat([df, pd.DataFrame(columns=param_names, data=model.Ainv)], axis=1)
    df.to_csv("params/"+strain+".csv", index=False)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Total samples: 99, Initial regularization: 1.00e-03
Loss: 91.057, Residuals: -0.922
Loss: 72.280, Residuals: -0.948
Loss: 64.886, Residuals: -1.413
Loss: 63.090, Residuals: -1.464
Loss: 60.330, Residuals: -1.476
Loss: 59.670, Residuals: -1.371
Loss: 58.496, Residuals: -1.399
Loss: 58.273, Residuals: -1.373
Loss: 57.871, Residuals: -1.385
Loss: 57.250, Residuals: -1.396
Loss: 57.222, Residuals: -1.393
Loss: 57.003, Residuals: -1.391
Loss: 56.965, Residuals: -1.366
Loss: 56.892, Residuals: -1.371
Loss: 56.761, Residuals: -1.380
Loss: 56.757, Residuals: -1.382
Loss: 56.612, Residuals: -1.392
Loss: 56.609, Residuals: -1.386
Loss: 56.495, Residuals: -1.394
Loss: 56.494, Residuals: -1.394
Optimization terminated successfully.
Evidence -167.567
Updating hyper-parameters...
Total samples: 99, Updated regularization: 1.02e+00
Loss: 362.669, Residuals: -1.395
Optimization terminated successfully.
Evidence 3652.242
Updating hyper-parameters...
Total samples: 99, Updated regularization: 1.06e+00
L

In [ ]:
df